In [1]:
# !pip install geojson
# !pip install folium
# !pip install geopandas
# !pip install pyogrio

In [2]:
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
import json
import folium
import geopandas as gpd
import pyogrio
from shapely.ops import split
from shapely.geometry import MultiLineString, LineString, Point, Polygon, MultiPolygon
import pandas as pd
import numpy as np
from folium.plugins import MarkerCluster
import os
import requests
import json
from utils_data import get_contours_city_ign, add_area_gdf, get_data_carroyee_city 
from utils_data import get_euclidian_adjacency_from_df, get_adjacency_from_gdf, get_vehicles_city, get_df_OD_city, get_contours_city_simplifie, get_centroids_communes, restrict_area, get_routing, get_traffic_demand_per_carreau, compute_routes_OD

In [3]:
code_insee=44055
crs_meters = 'EPSG:2154'
crs_angles = 'EPSG:4326'
waiting_time_api=0.1
restricted_distance_km = 150

avg_dist_hab = 30
conso_kwh_km = 0.2
avg_demand_hab = avg_dist_hab * conso_kwh_km
ratio_ev_france = 0.03 

In [4]:
data_dir = '../data'
dir_ev = f"{data_dir}/vehicules"
file_vp = f"{dir_ev}/parc_vp_com_2011_2024.xlsx"
file_vul = f"{dir_ev}/parc_vul_com_2011_2024.xlsx"
file_pl = f"{dir_ev}/parc_pl_com2011_2024.xlsx"
file_tcp = f"{dir_ev}/parc_tcp_commune_2024.xlsx"
files_vehicules = {'vp': file_vp, 'vul': file_vul, 'pl': file_pl, 'tcp': file_tcp}

processed_dir = f'{data_dir}/processed'
preprocessed_dir = f'{data_dir}/preprocessed'
os.makedirs(processed_dir, exist_ok=True)
os.makedirs(preprocessed_dir, exist_ok=True)

preprocessed_file_vp = f"{preprocessed_dir}/vp_{code_insee}.xlsx"
preprocessed_file_vul = f"{preprocessed_dir}/vul_{code_insee}.xlsx"
preprocessed_file_pl = f"{preprocessed_dir}/pl_{code_insee}.xlsx"
preprocessed_file_tcp = f"{preprocessed_dir}/tcp_{code_insee}.xlsx"
processed_files_vehicules = {'vp': preprocessed_file_vp, 'vul': preprocessed_file_vul, 'pl': preprocessed_file_pl, 'tcp': preprocessed_file_tcp}
export_file_vehicles = f"{processed_dir}/df_vehicles_{code_insee}_2024.csv"

file_ign = f'{data_dir}/BDCARTO/44_Loire_Atlantique/data.gpkg'
file_carroye = f'{data_dir}/Filosofi2017_carreaux_200m_met.gpkg'

export_file_OD_routes = f"{processed_dir}/processed_OD_{code_insee}.parquet"
export_file_df_adjacency_gpkg=f"{processed_dir}/gdf_city_200_with_dist_{code_insee}.gpkg"
export_file_np_adjacency=f"{processed_dir}/adjacency_matrix_200_{code_insee}.npy"


In [5]:
file_OD = f'{data_dir}/RP2020_MOBPRO_csv/FD_MOBPRO_2020.csv'
file_contours_simplifies = f'{data_dir}/communes-version-simplifiee.geojson'

In [6]:
gdf_contours_city, geometry_contours_city = get_contours_city_ign(file_ign, code_insee, crs=crs_angles)
center_lon, center_lat = geometry_contours_city.centroid.coords[0]

In [ ]:
gdf_carroye_city = get_data_carroyee_city(file_carroye, geometry_contours_city, crs=crs_angles)

In [8]:
cols = ['Idcar_200m', 'I_est_200', 'Ind', 'geometry', 'Men_coll', 'Men_mais', 'Ind_snv']
gdf_carroye_city = gdf_carroye_city[cols]
gdf_carroye_city = gdf_carroye_city.rename(columns={'Idcar_200m':'id', 'I_est_200':'estimated', 'Ind':'population', 'Ind_snv':'niveau_de_vie'}) 
gdf_carroye_city = add_area_gdf(gdf_carroye_city, crs_meters=crs_meters, crs_angles=crs_angles)

gdf_carroye_city['density_km2'] = gdf_carroye_city['population'] / gdf_carroye_city['area_km2']
gdf_carroye_city['ratio_appart'] = gdf_carroye_city['Men_coll'] / (gdf_carroye_city['Men_coll'] + gdf_carroye_city['Men_mais'])
gdf_carroye_city['niveau_de_vie_moyen'] = gdf_carroye_city['niveau_de_vie'] / gdf_carroye_city['population']

gdf_carroye_city.drop(columns=['area_km2', 'Men_coll', 'Men_mais', 'niveau_de_vie'], inplace=True)

In [9]:
gdf_carroye_city.head(2)

,id,estimated,population,geometry,density_km2,ratio_appart,niveau_de_vie_moyen
0,CRS3035RES200mN2761400E3391600,0,31.5,"POLYGON ((-2.3372 47.26715, -2.33762 47.26892,...",788.861822,0.15,29316.704762
1,CRS3035RES200mN2761400E3391800,0,26.0,"POLYGON ((-2.33459 47.26745, -2.33501 47.26922...",651.123954,0.00,23425.138462


# Population demand

In [10]:
pop_city = gdf_carroye_city.population.sum() # population de la ville à partir des carreaux
df_vehicules = get_vehicles_city(code_insee, files_vehicules, processed_files_vehicules, export_file_vehicles)
ratio_ev_pop = ((df_vehicules['NB_EL']+df_vehicules['NB_RECHARGEABLE'])/pop_city).iloc[0]
gdf_carroye_city['demand_pop_kWh'] = gdf_carroye_city['population'] * ratio_ev_pop * gdf_carroye_city['ratio_appart'] * avg_demand_hab
gdf_carroye_city.drop(columns=['ratio_appart'], inplace=True) 

File already exists


In [11]:
gdf_carroye_city.head(2)

,id,estimated,population,geometry,density_km2,niveau_de_vie_moyen,demand_pop_kWh
0,CRS3035RES200mN2761400E3391600,0,31.5,"POLYGON ((-2.3372 47.26715, -2.33762 47.26892,...",788.861822,29316.704762,1.05137
1,CRS3035RES200mN2761400E3391800,0,26.0,"POLYGON ((-2.33459 47.26745, -2.33501 47.26922...",651.123954,23425.138462,0.00000


In [12]:
gdf_carroye_city.demand_pop_kWh.sum()

1628.4055421958844

# Trafic demand

In [13]:
df_traffic_city = get_df_OD_city(file_OD, code_insee)
gdf_contours_city, geometry_contours_city = get_contours_city_simplifie(file_contours_simplifies, code_insee)

c:\Users\eliot\Documents\M2DS\capstone_git\capstone-vinci\demand_estimation\utils_data.py:574: DtypeWarning: Columns (0,1,2,3,13,14,15,17,21,23,26,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mob_full = pd.read_csv(file_OD, sep=';')
c:\Users\eliot\Documents\M2DS\capstone_git\capstone-vinci\demand_estimation\utils_data.py:586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mob['transport_name'] = df_mob['TRANS'].apply(lambda x: dict_transport.get(x, 'default_value'))


In [31]:
# Commute centroid in meters
gdf_contours_city = gdf_contours_city.to_crs(crs_meters)
gdf_contours_city['centroid'] = gdf_contours_city.geometry.centroid

# Put back in angles
gdf_contours_city = gdf_contours_city.to_crs(crs_angles)
gdf_contours_city['centroid'] = gdf_contours_city['centroid'].to_crs(crs_angles)
centroid_city = gdf_contours_city.iloc[0].centroid

# Add centroids features in each row
gdf_od_full = get_centroids_communes(file_contours_simplifies, df_traffic_city, centroid_city)

# Keep only trips within a limited area
gdf_od_filtered, restricted_area = restrict_area(gdf_od_full, restricted_distance_km)

# Get routes and distances using API
if not os.path.exists(export_file_OD_routes):
    gdf_od_filtered = compute_routes_OD(gdf_od_filtered, export_file_OD_routes, waiting_time_api=0.1, crs_angles='EPSG:4326')
else:
    # gdf_od_filtered = gpd.read_file(export_file_OD_routes)
    gdf_od_filtered = gpd.read_parquet(export_file_OD_routes)

In [32]:
gdf_carroye_city = get_traffic_demand_per_carreau(gdf_od_filtered, gdf_carroye_city, ratio_ev_france, conso_kwh_km)

In [33]:
gdf_carroye_city.demand_traffic_kWh.sum()

120.59373900000001

In [34]:
gdf_carroye_city['total_demand_kWh'] = gdf_carroye_city['demand_pop_kWh'] + gdf_carroye_city['demand_traffic_kWh']

In [35]:
gdf_carroye_city

,id,estimated,population,geometry,density_km2,niveau_de_vie_moyen,demand_pop_kWh,demand_traffic_kWh,total_demand_kWh
0,CRS3035RES200mN2761400E3391600,0,31.5,"POLYGON ((-2.3372 47.26715, -2.33762 47.26892,...",788.861822,29316.704762,1.051370,0.0,1.051370
1,CRS3035RES200mN2761400E3391800,0,26.0,"POLYGON ((-2.33459 47.26745, -2.33501 47.26922...",651.123954,23425.138462,0.000000,0.0,0.000000
2,CRS3035RES200mN2761400E3392000,1,2.0,"POLYGON ((-2.33197 47.26774, -2.3324 47.26952,...",50.086451,28876.200000,0.000000,0.0,0.000000
3,CRS3035RES200mN2761600E3391200,0,93.5,"POLYGON ((-2.34285 47.26832, -2.34327 47.27009...",2341.540973,24798.703743,11.284012,0.0,11.284012
4,CRS3035RES200mN2761600E3391400,0,72.0,"POLYGON ((-2.34023 47.26862, -2.34066 47.27039...",1803.111518,27235.497222,0.890049,0.0,0.890049
...,...,...,...,...,...,...,...,...,...
426,CRS3035RES200mN2766200E3390800,1,2.0,"POLYGON ((-2.35786 47.30847, -2.35828 47.31024...",50.085491,22899.050000,0.000000,0.0,0.000000
427,CRS3035RES200mN2766200E3391000,1,3.0,"POLYGON ((-2.35524 47.30877, -2.35567 47.31054...",75.128225,27568.400000,0.000000,0.0,0.000000
428,CRS3035RES200mN2766400E3391000,1,2.0,"POLYGON ((-2.35567 47.31054, -2.35609 47.31231...",50.085440,27568.400000,0.000000,0.0,0.000000
429,CRS3035RES200mN2766600E3389000,1,3.0,"POLYGON ((-2.38223 47.30932, -2.38266 47.31109...",75.128205,18076.700000,0.060685,0.0,0.060685


# Distances

In [36]:
# gdf_city_200_with_dist, adjacency_matrix = get_adjacency_from_gdf(gdf_carroye_city_kept, delay=waiting_time_api, crs_angles=crs_angles, crs_meters=crs_meters)
gdf_city_200_with_dist, adjacency_matrix = get_euclidian_adjacency_from_df(gdf_carroye_city, crs_angles=crs_angles, crs_meters=crs_meters)

In [37]:
gdf_city_200_with_dist.head()

,id,estimated,population,geometry,density_km2,niveau_de_vie_moyen,demand_pop_kWh,demand_traffic_kWh,total_demand_kWh,centroid,...,dist_423,dist_424,dist_425,dist_426,dist_427,dist_428,dist_429,dist_430,longitude,latitude
0,CRS3035RES200mN2761400E3391600,0,31.5,"POLYGON ((-2.3372 47.26715, -2.33762 47.26892,...",788.861822,29316.704762,1.051370,0.0,1.051370,POINT (-2.3361 47.26818),...,5.534841,5.436998,4.884685,4.848552,4.820473,5.018388,5.790279,5.750147,-2.336104,47.268182
1,CRS3035RES200mN2761400E3391800,0,26.0,"POLYGON ((-2.33459 47.26745, -2.33501 47.26922...",651.123954,23425.138462,0.000000,0.0,0.000000,POINT (-2.33349 47.26848),...,5.638107,5.534847,4.928702,4.884691,4.848558,5.045333,5.882190,5.801296,-2.333492,47.268481
2,CRS3035RES200mN2761400E3392000,1,2.0,"POLYGON ((-2.33197 47.26774, -2.3324 47.26952,...",50.086451,28876.200000,0.000000,0.0,0.000000,POINT (-2.33088 47.26878),...,5.746497,5.638112,4.980388,4.928708,4.884698,5.080036,5.979397,5.858848,-2.330880,47.268780
3,CRS3035RES200mN2761600E3391200,0,93.5,"POLYGON ((-2.34285 47.26832, -2.34327 47.27009...",2341.540973,24798.703743,11.284012,0.0,11.284012,POINT (-2.34175 47.26936),...,5.167523,5.078366,4.622676,4.601897,4.589769,4.788994,5.440463,5.471568,-2.341752,47.269356
4,CRS3035RES200mN2761600E3391400,0,72.0,"POLYGON ((-2.34023 47.26862, -2.34066 47.27039...",1803.111518,27235.497222,0.890049,0.0,0.890049,POINT (-2.33914 47.26965),...,5.262799,5.167529,4.651996,4.622682,4.601903,4.800585,5.523716,5.510797,-2.339140,47.269655


In [38]:
# Clean columns 			
cols = ['id', 'geometry', 'population', 'density_km2', 'niveau_de_vie_moyen', 'demand_pop_kWh', 'demand_traffic_kWh', 'total_demand_kWh']
cols += [f'dist_{i}' for i in range(len(gdf_city_200_with_dist))]

gdf_city_with_dist = gdf_city_200_with_dist[cols]
gdf_city_with_dist.to_file(export_file_df_adjacency_gpkg, driver="GPKG")
print(f"Dataframe saved at {export_file_df_adjacency_gpkg}")

np.save(export_file_np_adjacency, adjacency_matrix)
print(f"Adjacency matrix alone saved at {export_file_np_adjacency}")

Dataframe saved at ../data/processed/gdf_city_200_with_dist_44055.gpkg
Adjacency matrix alone saved at ../data/processed/adjacency_matrix_200_44055.npy


In [39]:
gdf_city_with_dist = gpd.read_file(export_file_df_adjacency_gpkg)

## Toute la demande

In [ ]:
import branca

m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add the city boundary contour
folium.GeoJson(
    geometry_contours_city,
    name="La Baule",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
).add_to(m)

# Create a colormap based on the number of people in each carreau
min_people = gdf_carroye_city['total_demand_kWh'].min()  # Minimum number of people
max_people = gdf_carroye_city['total_demand_kWh'].max()  # Maximum number of people
colormap = branca.colormap.linear.YlOrRd_09.scale(min_people, max_people)  # Yellow to red color scale

# Add the colormap legend to the map
colormap.add_to(m)

# Iterate through each row in the GeoDataFrame and plot each feature
for _, row in gdf_carroye_city.iterrows():
    # Extract geometry and properties for each feature
    feature = row['geometry']
    demand = row['total_demand_kWh']  # Assuming this column exists
    # num_menages = row['Men']  # Assuming this column exists
    cond = row['estimated'] == 1

    # Create a popup displaying the number of people
    text = f"Demand: {demand}kWh"
    popup = folium.Popup(text, max_width=300)
    
    # print(num_people)
    

    # Style each feature based on the number of people
    folium.GeoJson(
        feature,
        name=f"Carreau {row.id}",
        popup=popup,
        style_function=lambda x, demand=demand: {
                'fillColor': colormap(demand),  # Color based on the number of people
                'color': 'black',  # Border color
                'weight': 2,  # Border width
                'fillOpacity': 0.7  # Fill opacity
            }
    ).add_to(m)

# Return the map to visualize
m

## Demande de population

In [26]:
import branca


m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add the city boundary contour
folium.GeoJson(
    geometry_contours_city,
    name="La Baule",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
).add_to(m)

# Create a colormap based on the number of people in each carreau
min_people = gdf_carroye_city['demand_pop_kWh'].min()  # Minimum number of people
max_people = gdf_carroye_city['demand_pop_kWh'].max()  # Maximum number of people
colormap = branca.colormap.linear.YlOrRd_09.scale(min_people, max_people)  # Yellow to red color scale

# Add the colormap legend to the map
colormap.add_to(m)

# Iterate through each row in the GeoDataFrame and plot each feature
for _, row in gdf_carroye_city.iterrows():
    # Extract geometry and properties for each feature
    feature = row['geometry']
    demand = row['demand_pop_kWh']  # Assuming this column exists
    # num_menages = row['Men']  # Assuming this column exists
    cond = row['estimated'] == 1

    # Create a popup displaying the number of people
    text = f"Demand: {demand}kWh"
    popup = folium.Popup(text, max_width=300)
    
    # Style each feature based on the number of people
    folium.GeoJson(
        feature,
        name=f"Carreau {row.id}",
        popup=popup,
        style_function=lambda x, demand=demand: {
                'fillColor': colormap(demand),  # Color based on the number of people
                'color': 'black',  # Border color
                'weight': 2,  # Border width
                'fillOpacity': 0.7  # Fill opacity
            }
    ).add_to(m)

# Return the map to visualize
m

## Demande de trafic

In [27]:
import branca


m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add the city boundary contour
folium.GeoJson(
    geometry_contours_city,
    name="La Baule",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
).add_to(m)

# Create a colormap based on the number of people in each carreau
min_people = gdf_carroye_city['demand_traffic_kWh'].min()  # Minimum number of people
max_people = gdf_carroye_city['demand_traffic_kWh'].max()  # Maximum number of people
colormap = branca.colormap.linear.YlOrRd_09.scale(min_people, max_people)  # Yellow to red color scale

# Add the colormap legend to the map
colormap.add_to(m)

# Iterate through each row in the GeoDataFrame and plot each feature
for _, row in gdf_carroye_city.iterrows():
    # Extract geometry and properties for each feature
    feature = row['geometry']
    demand = row['demand_traffic_kWh']  # Assuming this column exists
    # num_menages = row['Men']  # Assuming this column exists
    cond = row['estimated'] == 1

    # Create a popup displaying the number of people
    text = f"Demand: {demand}kWh"
    popup = folium.Popup(text, max_width=300)
    
    # print(num_people)
    

    # Style each feature based on the number of people
    folium.GeoJson(
        feature,
        name=f"Carreau {row.id}",
        popup=popup,
        style_function=lambda x, demand=demand: {
                'fillColor': colormap(demand),  # Color based on the number of people
                'color': 'black',  # Border color
                'weight': 2,  # Border width
                'fillOpacity': 0.7  # Fill opacity
            }
    ).add_to(m)

# Return the map to visualize
m